In [0]:
from pyspark.sql import functions as F

In [0]:
schema_path = "dbfs:/FileStore/inventory_analytics"

In [0]:
hourly_purchase_rate = spark.read.format("delta").load(f"{schema_path}/gold/purchase_count")
click_ratio = spark.read.foramt("delta").load(f"{schema_path}/gold/click_ratio")
cart_ratio = spark.read.foramt("delta").load(f"{schema_path}/gold/cart_ratio")

In [0]:
adjustment_factor = cart_ratio.join(click_ratio, ["pin_code", "product_id", "hour_of_day"], "inner")\
    .withColumn("adjustment_factor",
                1 + (0.7*(cart_ratio.cart_ratio-1)) + (0.3 * (click_ratio.click_ratio - 1)))

In [0]:
product_forcast_next_24_hour = hourly_purchase_rate\
    .join(adjustment_factor,["pin_code", "product_id", "hour_of_day"], "left")\
    .withColumn("forcasted_count", 
                weighted_hourly_avg_purchase_next_24_hour.weighted_avg_purchase * F.coalesce(adjustment_factor.adjustment_factor, F.lit(1.0)))

In [0]:
product_forcast_next_24_hour.show(truncate=False)